<a href="http://esprit.tn"><img src = "https://upload.wikimedia.org/wikipedia/commons/f/ff/Logo_ESPRIT_Ariana.jpg" width = 300, align = "center"></a>



<h1 align=center><font size = 5>Lab 9: Supervised learning -  Artificial Neural Networks (ANN) </font></h1>

Estimated duration: 90 min

KEBOU DJEUFO Laurel Toussaint 4NIDS2

## LAB 9: IDS BASED ON MACHINE LEARNING TECHNIQUES

## Goal

We aim through this lab to build a robust intrusion detection system (IDS) based on a set of machine learning techinques.
Various techniques will be explored and experimented to assess IDS robustness according to two criteria:
    - maximizing the number of atatcks  detected 
    - reducing the number of false positive alerts
The experiemtal study will cover the following methods:
    - ANN
    - Decision trees / random forest
    - SVM

## Part I: Data loading and preparation 

Load the data preprare columns for ANN as done in the mini_lab9

In [13]:
import numpy as np
import pandas as pnd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, zero_one_loss)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)


In [18]:
import datetime
t=datetime.datetime.now()
print(datetime.datetime.now()- t)

def init_chrono():
    return datetime.datetime.now()

def elapsed_time(t):
    if t:
        delta = init_chrono() - t
        print ('Elapsed time', delta)
        return delta 
    
# test
t = init_chrono()
elapsed_time(t)

0:00:00.000073
Elapsed time 0:00:00.000034


datetime.timedelta(microseconds=34)

In [3]:
print(sk.__version__)

0.19.2


In [32]:
data_path = './'
traindata = pnd.read_csv(data_path+'train.csv', header=None)
testdata = pnd.read_csv(data_path+'test.csv', header=None)

print('train_data', traindata.shape, '\n', 'test_data:', testdata.shape)

train_data (494021, 42) 
 test_data: (311029, 42)


# Part II: Neural networks 

1) Build 3 neural networks with respecitvely 2 hidden layers, one hidden layer, and a learning rate equal to .5

In [5]:
import sklearn.neural_network as nn

In [6]:
def show_dist(df_column, tag = 'Column'):
    if not df_column.empty: 
        dist = df_column.value_counts()
        print("%s distribution:\n" %tag, dist)

#Test
show_dist(traindata[41], tag="train dataset before ")  

train dataset before  distribution:
 smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: 41, dtype: int64


In [7]:
def get_categorical(dataframe):
    types = dataframe.dtypes
    i = 0
    lst_cat=[]
    for var in types:
        if var == 'object':
            print(i)
            lst_cat.append(i)
        i=i+1
    return lst_cat

def handle_categorical(dataframe):
    if type(dataframe) != pnd.DataFrame:
        return None 

    lst= get_categorical(dataframe)
    
    print('%d variables are categorical %s ' %(len(lst), lst))
    for var_cat in lst:
        try:
            enc = sk.preprocessing.label.LabelEncoder()
            show_dist(dataframe[var_cat], 'Var[' + str(var_cat) + '] (before)')
            dataframe[var_cat] = enc.fit_transform(dataframe[var_cat])
            show_dist(dataframe[var_cat], 'Var[' + str(var_cat) + '] (after)')
        except Exception as e:
            print('Exception: Unable to convert categorical variable ', var_cat, e)
            
    #return dataframe
#handle_categorical(T)

In [33]:
traindata[41]=traindata[41].map(lambda x: 0 if x=='normal.' else 1)
testdata[41]=testdata[41].map(lambda x: 0 if x=='normal.' else 1)

In [9]:
show_dist(traindata[41], 'Train_dataset:')
show_dist(testdata[41], 'Test_dataset:')

Train_dataset: distribution:
 1    396743
0     97278
Name: 41, dtype: int64
Test_dataset: distribution:
 1    250436
0     60593
Name: 41, dtype: int64


In [34]:
handle_categorical(testdata)
handle_categorical(traindata)

1
2
3
3 variables are categorical [1, 2, 3] 
Var[1] (before) distribution:
 icmp    164969
tcp     119357
udp      26703
Name: 1, dtype: int64
Var[1] (after) distribution:
 0    164969
1    119357
2     26703
Name: 1, dtype: int64
Var[2] (before) distribution:
 ecr_i          164352
private         78510
http            41237
smtp             8268
pop_3            3972
domain_u         3160
ftp_data         2223
other            2185
telnet           2077
ftp               837
eco_i             547
imap4             333
finger            291
sunrpc            195
auth              150
time               82
ntp_u              80
echo               69
domain             68
link               66
bgp                66
netstat            65
gopher             65
remote_job         65
netbios_ns         64
ctf                63
urp_i              63
nntp               62
uucp               62
iso_tsap           62
                ...  
kshell             60
name               60
netbios_dgm 

In [35]:
print(traindata.shape, testdata.shape)
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]

(494021, 42) (311029, 42)


In [14]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)


scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)

First Model Hidden Layer 2

In [15]:
model1 = nn.MLPClassifier(activation='logistic' ,hidden_layer_sizes=(15,20), max_iter=10,solver='sgd', verbose=True)

In [19]:
t=init_chrono()
model1.fit(traindata, trainlabel)
print(model1)
elapsed_time(t)

Iteration 1, loss = 0.49829577
Iteration 2, loss = 0.49500504
Iteration 3, loss = 0.49394134
Iteration 4, loss = 0.49223700
Iteration 5, loss = 0.48915807
Iteration 6, loss = 0.48283069
Iteration 7, loss = 0.46770396
Iteration 8, loss = 0.42500888
Iteration 9, loss = 0.31346921
Iteration 10, loss = 0.17769351
MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)
Elapsed time 0:00:16.287574


/home/aminos/esprit/4nids/decision/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


datetime.timedelta(seconds=16, microseconds=287574)

In [20]:
# make predictions
expected = testlabel
predicted = model1.predict(testdata)
np.savetxt('./predictedDT1.txt', predicted, fmt='%01d')

In [21]:
# summarize the fit of the model
avg= 'macro'
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted, average=avg)
precision = precision_score(expected, predicted , average=avg)
f1 = f1_score(expected, predicted , average=avg)

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tnr=float((cm[0][0])/np.sum(cm[0]))
fpr=float((cm[0][1])/np.sum(cm[0]))
tpr = float((cm[1][1])/np.sum(cm[1]))
fnr = float((cm[1][0])/np.sum(cm[1]))
print("True Positive Rate: %.3f" %tpr)
print("False Positve Rate: %.3f" %fpr)
print("True Negative Rate: %.3f" %tnr)
print("False Negative Rate: %.3f" %fnr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)

print("***************************************************************")
elapsed_time(t)

[[ 50411  10182]
 [ 13959 236477]]
True Positive Rate: 0.944
False Positve Rate: 0.168
True Negative Rate: 0.832
False Negative Rate: 0.056
Accuracy
0.922
precision
0.871
recall
0.888
f-score
0.879
***************************************************************
Elapsed time 0:00:25.174956


datetime.timedelta(seconds=25, microseconds=174956)

Second Model hidden layer 1

In [22]:
model2 = nn.MLPClassifier(activation='logistic' ,hidden_layer_sizes=(20,), max_iter=10,solver='sgd', verbose=True)

In [23]:
t=init_chrono()
model2.fit(traindata, trainlabel)
print(model2)
elapsed_time(t)

Iteration 1, loss = 0.47923913
Iteration 2, loss = 0.39825907
Iteration 3, loss = 0.26632786
Iteration 4, loss = 0.16678754
Iteration 5, loss = 0.11786571
Iteration 6, loss = 0.09250247
Iteration 7, loss = 0.07789792
Iteration 8, loss = 0.06882631
Iteration 9, loss = 0.06285006
Iteration 10, loss = 0.05872717
MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)
Elapsed time 0:00:12.312365


/home/aminos/esprit/4nids/decision/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


datetime.timedelta(seconds=12, microseconds=312365)

In [24]:
# make predictions
expected = testlabel
predicted = model2.predict(testdata)
np.savetxt('./predictedDT2.txt', predicted, fmt='%01d')

In [25]:
# summarize the fit of the model
avg= 'macro'
accuracy_mod2 = accuracy_score(expected, predicted)
recall_mod2 = recall_score(expected, predicted, average=avg)
precision_mod2 = precision_score(expected, predicted , average=avg)
f1_mod2 = f1_score(expected, predicted , average=avg)

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tnr_mod2=float((cm[0][0])/np.sum(cm[0]))
fpr_mod2=float((cm[0][1])/np.sum(cm[0]))
tpr_mod2 = float((cm[1][1])/np.sum(cm[1]))
fnr_mod2 = float((cm[1][0])/np.sum(cm[1]))
print("True Positive Rate: %.3f" %tpr_mod2)
print("False Positve Rate: %.3f" %fpr_mod2)
print("True Negative Rate: %.3f" %tnr_mod2)
print("False Negative Rate: %.3f" %fnr_mod2)
print("Accuracy")
print("%.3f" %accuracy_mod2)
print("precision")
print("%.3f" %precision_mod2)
print("recall")
print("%.3f" %recall_mod2)
print("f-score")
print("%.3f" %f1_mod2)

print("***************************************************************")
elapsed_time(t)

[[ 57441   3152]
 [ 23547 226889]]
True Positive Rate: 0.906
False Positve Rate: 0.052
True Negative Rate: 0.948
False Negative Rate: 0.094
Accuracy
0.914
precision
0.848
recall
0.927
f-score
0.878
***************************************************************
Elapsed time 0:02:34.906664


datetime.timedelta(seconds=154, microseconds=906664)

Third Model learning_rate_init=0.5

In [26]:
model3 = nn.MLPClassifier(activation='logistic' ,hidden_layer_sizes=(5,), max_iter=10,solver='sgd', verbose=True, learning_rate='constant', learning_rate_init=0.5)

In [27]:
t=init_chrono()
model3.fit(traindata, trainlabel)
print(model3)
elapsed_time(t)

Iteration 1, loss = 0.04008824
Iteration 2, loss = 0.02756512
Iteration 3, loss = 0.02532387
Iteration 4, loss = 0.02297718
Iteration 5, loss = 0.02049638
Iteration 6, loss = 0.01891104
Iteration 7, loss = 0.01814683
Iteration 8, loss = 0.01757086
Iteration 9, loss = 0.01725618
Iteration 10, loss = 0.01689181
MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5,), learning_rate='constant',
       learning_rate_init=0.5, max_iter=10, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)
Elapsed time 0:00:09.752780


/home/aminos/esprit/4nids/decision/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


datetime.timedelta(seconds=9, microseconds=752780)

In [28]:
# make predictions
expected = testlabel
predicted = model3.predict(testdata)
np.savetxt('./predictedDT3.txt', predicted, fmt='%01d')

In [29]:
# summarize the fit of the model
avg= 'macro'
accuracy_mod3 = accuracy_score(expected, predicted)
recall_mod3 = recall_score(expected, predicted, average=avg)
precision_mod3 = precision_score(expected, predicted , average=avg)
f1_mod3 = f1_score(expected, predicted , average=avg)

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tnr_mod3=float((cm[0][0])/np.sum(cm[0]))
fpr_mod3=float((cm[0][1])/np.sum(cm[0]))
tpr_mod3 = float((cm[1][1])/np.sum(cm[1]))
fnr_mod3 = float((cm[1][0])/np.sum(cm[1]))
print("True Positive Rate: %.3f" %tpr_mod3)
print("False Positve Rate: %.3f" %fpr_mod3)
print("True Negative Rate: %.3f" %tnr_mod3)
print("False Negative Rate: %.3f" %fnr_mod3)
print("Accuracy")
print("%.3f" %accuracy_mod3)
print("precision")
print("%.3f" %precision_mod3)
print("recall")
print("%.3f" %recall_mod3)
print("f-score")
print("%.3f" %f1_mod3)

print("***************************************************************")
elapsed_time(t)

[[ 58767   1826]
 [ 22839 227597]]
True Positive Rate: 0.909
False Positve Rate: 0.030
True Negative Rate: 0.970
False Negative Rate: 0.091
Accuracy
0.921
precision
0.856
recall
0.939
f-score
0.888
***************************************************************
Elapsed time 0:00:16.617169


datetime.timedelta(seconds=16, microseconds=617169)

2) Compare accuracy performance of the obtained trained model and IDS assessment criteria: 
    (I) detected attacks, 
    (ii) false positive rate 


In [30]:
def Compare(*args):
    max_model=args[0]
    for i in args[1:]:
        if i[1]>max_model[1]:
            max_model=i
        elif i[1]==max_model[1]:
            if i[2]<max_model[2]:
                max_model=i
    print("The best Model is "+max_model)
    
def compare(predicat, *args):
    max_m = args[0]
    for i in args[1:]:
        if predicat(i, max_m) != False:
            max_m = i
    print("Better is {}".format(max_m[0]))

def best_tpr_if_equal_least_fpr(x, y):
    if x[1] > y[1]:
        return True
    elif x[1] == y[1]:
        if x[2] > y[2]:
            return False
        else:
            return True
    else:
        return False
m1 = ("model1", tpr, fpr)
m2 = ("model2", tpr_mod2, fpr_mod2)
m3 = ("model3", tpr_mod3, fpr_mod3)

compare(best_tpr_if_equal_least_fpr, m1, m2, m3)

Better is model1


## Part III:  Decision trees

1) Build a decision tree for a binary classification problem.


2) Compare false positives to best model obtained with neural net resuults

In [36]:
print(traindata.shape, testdata.shape)
X_train = traindata.iloc[:,0:41]
y_train = traindata.iloc[:,41]
y_test = testdata.iloc[:,41]
X_test = testdata.iloc[:,0:41]

(494021, 42) (311029, 42)


In [37]:
testlabel = np.array(y_test)
trainlabel = np.array(y_train)

In [38]:
# Training, choose model by commenting/uncommenting clf=
t= init_chrono()
DTclf = DecisionTreeClassifier(criterion='entropy', splitter='best',
                             max_depth=None, min_samples_split=2,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features=None,
                             random_state=None,
                             max_leaf_nodes=None,
                             min_impurity_decrease=0.0,
                             class_weight=None,
                             presort=False)
print("Training model")

DTtrained_model= DTclf.fit(X_train, trainlabel)
elapsed_time(t)

Training model
Elapsed time 0:00:01.379895


datetime.timedelta(seconds=1, microseconds=379895)

In [39]:
# Predicting
print ("Predicting using the test set ")
y_pred = DTclf.predict(X_test)
print(y_pred.shape)
print(y_pred[:10])

Predicting using the test set 
(311029,)
[0 0 0 0 0 0 0 0 0 0]


In [40]:
#model evaluation
print ("Training Score:", DTtrained_model.score(X_train, trainlabel))

print ("Test_Score: ", DTtrained_model.score(X_test, testlabel))

Training Score: 0.9999979757945513
Test_Score:  0.9281578245115407


In [41]:
#from sklearn.metrics import f1_score
print('By_class:',f1_score(testlabel, y_pred, average=None ))
print('Global:', f1_score(testlabel, y_pred, average='macro' ))

By_class: [0.8418445  0.95352276]
Global: 0.8976836315449059


In [42]:
print ("Computing performance metrics")
results = confusion_matrix(testlabel, y_pred)
error = zero_one_loss(testlabel, y_pred)

print ("Confusion matrix:\n", results)
print ("Error: ", error)

Computing performance metrics
Confusion matrix:
 [[ 59470   1123]
 [ 21222 229214]]
Error:  0.07184217548845928


In [43]:
# changing variables to coordonate comparaison with other models
expected = testlabel
predicted = y_pred
np.savetxt('./predictedDT_Dec_Tree.txt', predicted, fmt='%01d')

In [44]:
# summarize the fit of the model
avg= 'macro'
accuracy_DTmod = accuracy_score(expected, predicted)
recall_DTmod = recall_score(expected, predicted, average=avg)
precision_DTmod = precision_score(expected, predicted , average=avg)
f1_DTmod = f1_score(expected, predicted , average=avg)

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tnr_DTmod=float((cm[0][0])/np.sum(cm[0]))
fpr_DTmod=float((cm[0][1])/np.sum(cm[0]))
tpr_DTmod = float((cm[1][1])/np.sum(cm[1]))
fnr_DTmod = float((cm[1][0])/np.sum(cm[1]))
print("True Positive Rate: %.3f" %tpr_DTmod)
print("False Positve Rate: %.3f" %fpr_DTmod)
print("True Negative Rate: %.3f" %tnr_DTmod)
print("False Negative Rate: %.3f" %fnr_DTmod)
print("Accuracy")
print("%.3f" %accuracy_DTmod)
print("precision")
print("%.3f" %precision_DTmod)
print("recall")
print("%.3f" %recall_DTmod)
print("f-score")
print("%.3f" %f1_DTmod)

print("***************************************************************")
elapsed_time(t)

[[ 59470   1123]
 [ 21222 229214]]
True Positive Rate: 0.915
False Positve Rate: 0.019
True Negative Rate: 0.981
False Negative Rate: 0.085
Accuracy
0.928
precision
0.866
recall
0.948
f-score
0.898
***************************************************************
Elapsed time 0:00:24.850196


datetime.timedelta(seconds=24, microseconds=850196)

In [45]:
#Comparing Decision Tree model with the best ANN model (model1)
m1 = ("ANN model1", tpr, fpr)
m2 = ("Decision Tree model", tpr_DTmod, fpr_DTmod)
compare(best_tpr_if_equal_least_fpr, m1, m2)
#Comparing FalsePositves

if fpr>fpr_DTmod:
    print("False Positive are more significant with ANN model1")
else:
    print("False Positive are more significant with Decision Tree model")

Better is ANN model1
False Positive are more significant with ANN model1


## Part IV: Support Vector Machines 

1)-Build an Ids based on  2 svm by selecting the more accurate one: 
    (1)  SVM with a linear kernel 
    (2)  SVM with an rbf kernel an rbf svm kernel.


2)-  Compare the accuracy of svm models, ANN and decision tree


3)- Comapre computaional cost


4)- Can you find a tradeoff between accuracy and training cost and select the best model?

In [46]:
SVMclf_linear=svm.SVC(kernel='linear')

In [ ]:
t= init_chrono()
print("Training model")
SVMclf_linear.fit(X_train,trainlabel)
elapsed_time(t)

Training model


 <hr>
Copyright &copy; Nov-2018 Dr.Amir Esseghir